In [40]:
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from tqdm import tqdm

In [2]:
from data import get_data

In [3]:
X_train, y_train, X_test, y_test = get_data(one_hot=True)

In [4]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(20000, 128, 128, 3) (20000, 2) (5000, 128, 128, 3) (5000, 2)


In [5]:
train_len, width, height, channel = X_train.shape
test_len = X_test.shape[0]
print(train_len, width, height, channel, test_len)

20000 128 128 3 5000


In [6]:
# 学习率
learning_rate = 0.001
# 迭代次数（批次）
n_epoch = 20
# 批次大小
batch_size = 128

In [7]:
# 输入大小
input_size = (width, height, channel)
# 输出大小
target_size = 2

In [8]:
# 输入占位符
X = tf.placeholder(tf.float32, [batch_size, width, height, channel])
# 输出占位符
y = tf.placeholder(tf.float32, [batch_size, target_size])

In [43]:
pitch_1 = tf.Variable(tf.random_normal([7, 7, 3, 64]), name='pitch_1')
pitch_1_bias = tf.Variable(tf.random_normal([64]), name='pitch_1_bias')

pitch_2 = tf.Variable(tf.random_normal([5, 5, 64, 128]), name='pitch_2')
pitch_2_bias = tf.Variable(tf.random_normal([128]), name='pitch_2_bias')

pitch_3 = tf.Variable(tf.random_normal([5, 5, 128, 256]), name='pitch_3')
pitch_3_bias = tf.Variable(tf.random_normal([256]), name='pitch_3_bias')

pitch_4 = tf.Variable(tf.random_normal([3, 3, 256, 512]), name='pitch_4')
pitch_4_bias = tf.Variable(tf.random_normal([512]), name='pitch_4_bias')

pitch_5 = tf.Variable(tf.random_normal([2, 2, 512, 512]), name='pitch_5')
pitch_5_bias = tf.Variable(tf.random_normal([512]), name='pitch_5_bias')

In [44]:
conv_1 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            X, pitch_1, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_1_bias,
        name='bias_add_1'
    ),
    name='relu_1'
)

In [45]:
print(conv_1.get_shape())

(128, 122, 122, 64)


In [46]:
maxpool_1 = tf.nn.max_pool(
    conv_1,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID',
    name='max_pool_1'
)

In [47]:
print(maxpool_1.get_shape())

(128, 61, 61, 64)


In [48]:
conv_2 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            maxpool_1, pitch_2, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_2_bias,
        name='bias_add_2'
    ),
    name='relu_2'
)

In [49]:
print(conv_2.get_shape())

(128, 57, 57, 128)


In [50]:
maxpool_2 = tf.nn.max_pool(
    conv_2,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID',
    name='max_pool_2'
)

In [51]:
print(maxpool_2.get_shape())

(128, 28, 28, 128)


In [52]:
conv_3 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            maxpool_2, pitch_3, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_3_bias,
        name='bias_add_3'
    ),
    name='relu_3'
)

In [53]:
maxpool_3 = tf.nn.max_pool(
    conv_3,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID',
    name='max_pool_3'
)

In [54]:
print(maxpool_3.get_shape())

(128, 12, 12, 256)


In [55]:
conv_4 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            maxpool_3, pitch_4, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_4_bias,
        name='bias_add_4'
    ),
    name='relu_4'
)

In [56]:
print(conv_4.get_shape())

(128, 10, 10, 512)


In [57]:
maxpool_4 = tf.nn.max_pool(
    conv_4,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID',
    name='max_pool_4'
)

In [58]:
print(maxpool_4.get_shape())

(128, 5, 5, 512)


In [59]:
conv_5 = tf.nn.relu(
    tf.nn.bias_add(
        tf.nn.conv2d(
            maxpool_4, pitch_5, strides=[1, 1, 1, 1], padding='VALID'
        ),
        pitch_5_bias,
        name='bias_add_5'
    ),
    name='relu_5'
)

In [60]:
print(conv_5.get_shape())

(128, 4, 4, 512)


In [61]:
maxpool_5 = tf.nn.max_pool(
    conv_5,
    ksize=[1, 2, 2, 1],
    strides=[1, 2, 2, 1],
    padding='VALID',
    name='max_pool_5'
)

In [62]:
print(maxpool_5.get_shape())

(128, 2, 2, 512)


In [63]:
flatten = tf.reshape(maxpool_5, [batch_size, -1])

In [64]:
print(flatten.get_shape())
dim = int(flatten.get_shape()[1])
print('dim is {}'.format(dim))

(128, 2048)
dim is 2048


In [65]:
weight_1 = tf.Variable(tf.random_normal([dim, 2048]), name='weight_1')
bias_1 = tf.Variable(tf.random_normal([2048]), name='bias_1')

weight_2 = tf.Variable(tf.random_normal([2048, target_size]), name='weight_2')
bias_2 = tf.Variable(tf.random_normal([target_size]), name='bias_2')

In [66]:
full_connect_1 = tf.nn.relu(
    tf.add(
        tf.matmul(flatten, weight_1, name='matmul_1'),
        bias_1,
        name='add_1'
    ),
    name='relu_5'
)

In [67]:
print(full_connect_1.get_shape())

(128, 2048)


In [68]:
full_connect_2 = tf.add(
    tf.matmul(full_connect_1, weight_2, name='matmul_2'),
    bias_2,
    name='add_2'
)

In [69]:
print(full_connect_2.get_shape())

(128, 2)


In [70]:
pred = full_connect_2

In [71]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        pred, y
    )
)

In [72]:
opt = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

In [73]:
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [74]:
init = tf.initialize_all_variables()

In [75]:
def batch_flow(inputs, targets, batch_size):
    """流动数据流"""
    flowed = 0
    total = len(inputs)
    while True:
        X_ret = []
        y_ret = []
        for i in range(total):
            X_ret.append(inputs[i])
            y_ret.append([targets[i]])
            if len(X_ret) == batch_size:
                flowed += batch_size
                X, y = np.array(X_ret), np.array(y_ret)
                y = y.reshape([batch_size, -1])
                yield X, y
                X_ret = []
                y_ret = []
            if flowed >= total:
                break
        if flowed >= total:
            break

In [76]:
for batch_x, batch_y in batch_flow(X_train, y_train, batch_size):
    print(batch_x.shape, batch_y.shape)
    break

(128, 128, 128, 3) (128, 2)


In [77]:
with tf.Session() as sess:
    sess.run(init)
    total = None
    for epoch in range(n_epoch):
        costs = []
        accs = []
        for batch_x, batch_y in tqdm(batch_flow(X_train, y_train, batch_size), total=total):
            sess.run(opt, feed_dict={X: batch_x, y: batch_y})
            c, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y})
            costs.append(c)
            accs.append(acc)
        print('epoch: {}, loss: {:.4f}, acc: {:.4f}'.format(epoch, np.mean(costs), np.mean(accs)))
        if total is None:
            total = len(costs)
    print('calculate test accuracy')
    costs = []
    accs = []
    for batch_x, batch_y in tqdm(batch_flow(X_test, y_test, batch_size)):
        sess.run(opt, feed_dict={X: batch_x, y: batch_y})
        c, acc = sess.run([cost, accuracy], feed_dict={X: batch_x, y: batch_y})
        costs.append(c)
        accs.append(acc)
    print('test loss: {:.4f}, acc: {:.4f}'.format(np.mean(costs), np.mean(accs)))
    print('Done')

157it [01:47,  1.48it/s]
  0%|          | 0/157 [00:00<?, ?it/s]

epoch: 0, loss: 205711638528.0000, acc: 0.5978


  0%|          | 0/157 [00:00<?, ?it/s]

epoch: 1, loss: 56825511936.0000, acc: 0.6529


  0%|          | 0/157 [00:00<?, ?it/s]

...

epoch: 18, loss: 1771269376.0000, acc: 0.9287


100%|██████████| 157/157 [01:05<00:00,  2.80it/s]
0it [00:00, ?it/s]

epoch: 19, loss: 1418382464.0000, acc: 0.9381
calculate test accuracy


40it [00:14,  2.82it/s]

test loss: 19454973952.0000, acc: 0.6951
Done


过拟合的很严重～～恩～～